# Projeto 1 - Automação de Processo - Aplicação de Mercado de Trabalho 

In [ ]:
#importar bibliotecas
import pandas as pd
import pathlib
import win32com.client as win32
import os

#importar as bases de dados
emails = pd.read_excel(r'./Bases de Dados/Emails.xlsx')
lojas = pd.read_csv(r'./Bases de Dados/Lojas.csv', encoding = 'latin1', sep = ';')
vendas = pd.read_excel(r'./Bases de Dados/Vendas.xlsx')
display(emails)
display(lojas)
display(vendas)

#incluir nome da loja em vendas
vendas = vendas.merge(lojas, on='ID Loja')
display(vendas)
dicionario_lojas = {}
for loja in lojas['Loja']:
    dicionario_lojas[loja] = vendas.loc[vendas['Loja'] == loja, :]
display(dicionario_lojas['Rio Mar Recife'])
display(dicionario_lojas['Shopping Vila Velha'])

#dia do indicador
dia_indicador = vendas['Data'].max()
print(dia_indicador)
print(f'{dia_indicador.day}/{dia_indicador.month}/{dia_indicador.year}')
print('{}/{}'.format(dia_indicador.day,dia_indicador.month))


#identificar se a pasta da loja já existe
caminho_backup =pathlib.Path(r'./Base de dados/Backup Arquivos Lojas')

arquivos_pasta_backup = caminho_backup.iterdir()

lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup]

for loja in dicionario_lojas:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()

#salvar dentro da pasta
    nome_arquivo = '{}_{}_{}.xlsx'.format(dia_indicador.month, dia_indicador.day, loja)
    local_arquivo = caminho_backup / loja / nome_arquivo
    dicionario_lojas[loja].to_excel(local_arquivo)


#definição de matas
meta_faturamento_ano = 1650000 
meta_faturamento_dia = 1000
meta_qtdeprodutos_ano = 120
meta_qtdeprodutos_dia = 4
meta_ticketmedio_ano = 500 
meta_ticketmedio_dia = 500

for loja in dicionario_lojas:
    vendas_loja = dicionario_lojas[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']==dia_indicador, :]

    #faturamento
    faturamento_ano = vendas_loja['Valor Final'].sum()
    #print(faturamento_ano)
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()
    #print(faturamento_dia)
    
    #diversidade de produto
    qtde_produtos_ano = len(vendas_loja['Produto'].unique())
    #print (qtde_produtos_ano)
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())
    #print (qtde_produtos_dia)
    
    #ticket medio
    valor_venda = vendas_loja.drop(colunas_existentes, axis=1).groupby('Código Venda').sum()
    ticket_medio_ano = valor_venda['Valor Final'].mean()
    #print(f"Ticket médio anual: {ticket_medio_ano}")
    
    valor_venda = vendas_loja_dia.drop(colunas_existentes, axis=1).groupby('Código Venda').sum()
    ticket_medio_dia = valor_venda['Valor Final'].mean()
    #print(f"Ticket médio diário: {ticket_medio_dia}")

#enviar o e-mail
    outlook =win32.Dispatch('outlook.application')
    
    nome = emails.loc[emails['Loja']==loja, 'Gerente'].values[0]
    mail = outlook.CreateItem(0)
    mail.To = emails.loc[emails['Loja']==loja, 'E-mail'].values[0]
    mail.Subject = f'OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}'
    #mail.HTMLBody = 'Texto do E-mail'
    
    if faturamento_dia >= meta_faturamento_dia:
        cor_fat_dia = 'green'
    else:
        cor_fat_dia = 'red'
    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = 'green'
    else:
        cor_fat_ano = 'red'
    if qtde_produtos_dia >= meta_ qtdeprodutos_dia:
        cor_qtde_dia ='green'
    else:
        cor_qtde_dia = 'red'
    if qtde_produtos_ano >= meta_ qtdeprodutos_ano:
        cor_qtde_ano ='green'
    else:
        cor_qtde_ano = 'red'
    if qtde_medio_dia >= meta_ticketmedio_dia:
        cor_ticket_dia = 'green'
    else:
        cor_ticket_dia ='red'
    if qtde_medio_ano >= meta_ticketmedio_ano:
        cor_ticket_ano = 'green'
    else:
        cor_ticket_ano ='red'
        
    mail.HTMLBody = f'''
    <p>Bom dia, {nome}</p>
    <p>O resltado de ontem <strong>({dia.indicador.day}/{dia_indicador.month})</strong> da <strong>Loja {loja}</strong> foi:</p>
    
    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_dia:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_dia:.2f}</td>
        <td style="text-align: center"><font color = "{cor_fat_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_dia}</td>
        <td style="text-align: center">{meta_qtdeprodutos_dia}</td>
        <td style="text-align: center"><font color = "{cor_qtde_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_dia:.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_dia:.2f}</td>
        <td style="text-align: center"><font color = "{cor_ticket_dia}">◙</font></td>
      </tr>
     </table>
    <br>
    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cenário Ano</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_ano:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_ano:.2f}</td>
        <td style="text-align: center"><font color = "{cor_fat_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_ano}</td>
        <td style="text-align: center">{meta_qtdeprodutos_ano}</td>
        <td style="text-align: center"><font color = "{cor_qtde_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_ano:.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_ano:.2f}</td>
        <td style="text-align: center"><font color = "{cor_ticket_ano}">◙</font></td>
      </tr>
     </table>
    
    <p>Seue em anexo a planilha com todos os dados para mais detalhes</p>
    
    <p>Qualquel dúvida estou à disposição.</p>
    <p>Att., ...... </p>
    '''
    
    #anexos
    attachment = pathlib.Path.cwd() / caminho_backup / loja / f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
    mail.Attachments.Add(str(attachment))
    
    mail.Send()
    print('E-mail da loja {} enviado'.format(loja))


# Define o diretório onde os arquivos vão ser salvos
diretorio = r'/Base de dados/Backup Arquivos Lojas'

# Certifique-se de que o diretório exista, se não, crie-o
if not os.path.exists(diretorio):
    os.makedirs(diretorio)

# Agrupar por nome da loja e somar o 'Valor Final'
faturamento_lojas = vendas.groupby('Loja')['Valor Final'].sum().reset_index()
faturamento_lojas_ano = faturamento_lojas.sort_values(by='Valor Final', ascending=False)

# Exibir a tabela final com o nome da loja e o faturamento
display(faturamento_lojas_ano)

# Criar o nome do arquivo com a data
nome_arquivo_ano = '{}_{}_Ranking Anual.xlsx'.format(dia_indicador.month, dia_indicador.day)

# Salvar o arquivo na pasta especificada
faturamento_lojas_ano.to_excel(os.path.join(diretorio, nome_arquivo_ano))

# Filtrar vendas do dia específico
vendas_dia = vendas.loc[vendas['Data'] == dia_indicador, :]
faturamento_lojas_dia = vendas_dia.groupby('Loja')['Valor Final'].sum().reset_index()
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by='Valor Final', ascending=False)

# Exibir o faturamento do dia
display(faturamento_lojas_dia)

# Criar o nome do arquivo com a data para o ranking diário
nome_arquivo_dia = '{}_{}_Ranking Dia.xlsx'.format(dia_indicador.month, dia_indicador.day)

# Salvar o arquivo na pasta especificada
faturamento_lojas_dia.to_excel(os.path.join(diretorio, nome_arquivo_dia))



#enviar o e-mail
outlook =win32.Dispatch('outlook.application')

mail = outlook.CreateItem(0)
mail.To = emails.loc[emails['Loja']=='Diretoria', 'E-mail'].values[0]
mail.Subject = f'Ranking Dia {dia_indicador.day}/{dia_indicador.month}'
mail.Body = f'''
Prezados, bom dia

Melhor loja do Dia em Faturamento: {faturamento_lojas_dia['Loja'].iloc[0]} com Faturamento R${faturamento_lojas_dia['Valor Final'].iloc[0]:.2f}
Pior loja do Dia em Faturamento: {faturamento_lojas_dia['Loja'].iloc[-1]} com Faturamento R${faturamento_lojas_dia['Valor Final'].iloc[-1]:.2f}

Melhor loja do Ano em Faturamento: {faturamento_lojas_ano['Loja'].iloc[0]} com Faturamento R${faturamento_lojas_ano['Valor Final'].iloc[0]:.2f}
Pior loja do Ano em Faturamento: {faturamento_lojas_ano['Loja'].iloc[-1]} com Faturamento R${faturamento_lojas_ano['Valor Final'].iloc[-1]:.2f}

Segue em anexo os rankings do ano e do dia de todas as lojas.

Qualquer dúvida estou à disposição.

Att., 
.........

'''

#anexos
attachment = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.month}_{dia_indicador.day}_Ranking Anual.xlsx'
mail.Attachments.Add(str(attachment))
attachment = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.month}_{dia_indicador.day}_Ranking Dia.xlsx'

mail.Send()
print('E-mail da Diretoria {} enviado')

